In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import tensorflow as tf
from pathlib import Path
from skimage import io

from tqdm import tqdm_notebook
from scipy.spatial.distance import hamming, cosine

%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# import keras library
import keras

# import Sequential from the keras models module
from keras import Sequential

# import Dense, Dropout, Flatten, Conv2D, MaxPooling2D from the keras layers module
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
def image_loader(image_path, image_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size, cv2.INTER_CUBIC)
    
    return image

In [3]:
def dataset_preprocessing(dataset_path, labels_file_path, image_size, image_paths_pickle):
    with open(labels_file_path, "r") as f:
        classes = f.read().split("\n")
       
    images = []
    labels = []
    image_paths = []
    print(classes)
    
    for image_name in os.listdir(dataset_path):
        try:
            image_path = os.path.join(dataset_path, image_name)
            images.append(image_loader(image_path, image_size))
            image_paths.append(image_path)
            
            for idx in range(len(classes)):
                if classes[idx] in image_name:
                    labels.append(idx)
                    break
                elif len(classes) == idx:
                    labels.append(len(classes))
                    break
        except:
            pass
        
    with open(image_paths_pickle + ".pickle", "wb") as f:
        pickle.dump(image_paths, f)
    
   # assert len(images) == len(labels)
    return np.array(images), np.array(labels)

In [4]:
dataset_path = "JPEGImages/"
label_path = "labels.txt"
image_size = (120,72)

In [5]:
X, y = dataset_preprocessing(dataset_path, label_path, image_size, "test_image_pickle")

['bus', 'bicycle', 'car', 'motorbike', 'pedestrian', 'trafficlight', 'trafficsignal', '']


In [6]:
X.shape

(106920, 72, 120, 3)

In [7]:
y.shape

(106920,)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 123)

In [9]:
# set model constants
num_classes = 1

# define model as Sequential
model = Sequential()

# first convolutional layer with 32 filters
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(72, 120, 3)))

# add a second 2D convolutional layer with 64 filters
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

In [10]:
# reduce dimensionality through max pooling
model.add(MaxPooling2D(pool_size=(2, 2)))

# third convolutional layer with 64 filters
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# add dropout to prevent over fitting
model.add(Dropout(0.25))
# necessary flatten step preceeding dense layer
model.add(Flatten())
# fully connected layer
model.add(Dense(128, activation='relu'))

# add additional dropout to prevent overfitting
model.add(Dropout(0.5))

# prediction layers
model.add(Dense(num_classes, activation='sigmoid', name='preds'))

# show model summary
model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 70, 118, 32)       896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 68, 116, 64)       18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 34, 58, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 56, 64)        36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 56, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 114688)            0         
________________________

In [11]:
model.compile(
    # set the loss as binary_crossentropy
    loss='binary_crossentropy',
    # set the optimizer as stochastic gradient descent
    optimizer=keras.optimizers.SGD(lr=0.001),
    # set the metric as accuracy
    metrics=['accuracy']
)

# mock-train the model using the first ten observations of the train and test sets
model.fit(
    X_train,
    y_train,
    epochs=5,
    verbose=1,
    validation_data=(X_test, y_test)
)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 80190 samples, validate on 26730 samples
Epoch 1/5


 6176/80190 [=>............................] - ETA: 13:53:35 - loss: 34.9908 - acc: 0.50 - ETA: 7:21:42 - loss: 10.7697 - acc: 0.2656 - ETA: 5:05:56 - loss: 4.5227 - acc: 0.177 - ETA: 4:03:20 - loss: -3.3336 - acc: 0.132 - ETA: 3:27:09 - loss: -7.4496 - acc: 0.112 - ETA: 2:58:03 - loss: -7.5366 - acc: 0.099 - ETA: 2:37:44 - loss: -9.9473 - acc: 0.089 - ETA: 2:23:05 - loss: -11.4440 - acc: 0.08 - ETA: 2:10:50 - loss: -14.3241 - acc: 0.07 - ETA: 2:00:56 - loss: -16.5285 - acc: 0.07 - ETA: 1:52:53 - loss: -17.0640 - acc: 0.06 - ETA: 1:46:11 - loss: -18.9634 - acc: 0.06 - ETA: 1:40:36 - loss: -20.0723 - acc: 0.05 - ETA: 1:35:44 - loss: -20.1687 - acc: 0.05 - ETA: 1:31:30 - loss: -20.5512 - acc: 0.05 - ETA: 1:27:51 - loss: -20.1698 - acc: 0.04 - ETA: 1:24:36 - loss: -20.9468 - acc: 0.04 - ETA: 1:21:43 - loss: -21.2500 - acc: 0.04 - ETA: 1:19:06 - loss: -21.5738 - acc: 0.04 - ETA: 1:16:45 - loss: -21.9648 - acc: 0.04 - ETA: 1:14:36 - loss: -22.3660 - acc: 0.03 - ETA: 1:12:39 - loss: -23.4327

18592/80190 [=====>........................] - ETA: 30:41 - loss: -26.7213 - acc: 0.01 - ETA: 30:40 - loss: -26.6948 - acc: 0.01 - ETA: 30:39 - loss: -26.6724 - acc: 0.01 - ETA: 30:38 - loss: -26.6437 - acc: 0.01 - ETA: 30:37 - loss: -26.6596 - acc: 0.01 - ETA: 30:35 - loss: -26.6767 - acc: 0.01 - ETA: 30:34 - loss: -26.6861 - acc: 0.01 - ETA: 30:33 - loss: -26.6539 - acc: 0.01 - ETA: 30:32 - loss: -26.6105 - acc: 0.01 - ETA: 30:31 - loss: -26.6024 - acc: 0.01 - ETA: 30:29 - loss: -26.6169 - acc: 0.01 - ETA: 30:28 - loss: -26.6225 - acc: 0.01 - ETA: 30:27 - loss: -26.6046 - acc: 0.01 - ETA: 30:26 - loss: -26.6003 - acc: 0.01 - ETA: 30:25 - loss: -26.5862 - acc: 0.01 - ETA: 30:24 - loss: -26.5561 - acc: 0.01 - ETA: 30:22 - loss: -26.5643 - acc: 0.01 - ETA: 30:21 - loss: -26.5688 - acc: 0.01 - ETA: 30:20 - loss: -26.5660 - acc: 0.01 - ETA: 30:19 - loss: -26.5582 - acc: 0.01 - ETA: 30:18 - loss: -26.5859 - acc: 0.01 - ETA: 30:16 - loss: -26.5721 - acc: 0.01 - ETA: 30:15 - loss: -26.5753 -

24800/80190 [========>.....................] - ETA: 27:20 - loss: -26.9877 - acc: 0.01 - ETA: 27:19 - loss: -26.9987 - acc: 0.01 - ETA: 27:18 - loss: -26.9926 - acc: 0.01 - ETA: 27:17 - loss: -27.0018 - acc: 0.01 - ETA: 27:16 - loss: -26.9693 - acc: 0.01 - ETA: 27:15 - loss: -26.9582 - acc: 0.01 - ETA: 27:14 - loss: -26.9581 - acc: 0.01 - ETA: 27:13 - loss: -26.9681 - acc: 0.01 - ETA: 27:12 - loss: -26.9612 - acc: 0.01 - ETA: 27:11 - loss: -26.9552 - acc: 0.01 - ETA: 27:10 - loss: -26.9728 - acc: 0.01 - ETA: 27:09 - loss: -26.9659 - acc: 0.01 - ETA: 27:08 - loss: -26.9927 - acc: 0.01 - ETA: 27:07 - loss: -26.9959 - acc: 0.01 - ETA: 27:06 - loss: -26.9957 - acc: 0.01 - ETA: 27:05 - loss: -26.9755 - acc: 0.01 - ETA: 27:04 - loss: -26.9762 - acc: 0.01 - ETA: 27:03 - loss: -26.9686 - acc: 0.01 - ETA: 27:02 - loss: -26.9785 - acc: 0.01 - ETA: 27:01 - loss: -26.9825 - acc: 0.01 - ETA: 27:00 - loss: -26.9857 - acc: 0.01 - ETA: 26:59 - loss: -26.9930 - acc: 0.01 - ETA: 26:58 - loss: -26.9821 -

31008/80190 [==========>...................] - ETA: 24:12 - loss: -27.0765 - acc: 0.01 - ETA: 24:11 - loss: -27.0788 - acc: 0.01 - ETA: 24:10 - loss: -27.0927 - acc: 0.01 - ETA: 24:09 - loss: -27.1027 - acc: 0.01 - ETA: 24:09 - loss: -27.0986 - acc: 0.01 - ETA: 24:08 - loss: -27.1009 - acc: 0.01 - ETA: 24:07 - loss: -27.0917 - acc: 0.01 - ETA: 24:06 - loss: -27.0998 - acc: 0.01 - ETA: 24:05 - loss: -27.0931 - acc: 0.01 - ETA: 24:04 - loss: -27.0923 - acc: 0.01 - ETA: 24:03 - loss: -27.0965 - acc: 0.01 - ETA: 24:02 - loss: -27.0962 - acc: 0.01 - ETA: 24:01 - loss: -27.0903 - acc: 0.01 - ETA: 24:00 - loss: -27.1172 - acc: 0.01 - ETA: 23:59 - loss: -27.1226 - acc: 0.01 - ETA: 23:58 - loss: -27.1009 - acc: 0.01 - ETA: 23:57 - loss: -27.1063 - acc: 0.01 - ETA: 23:56 - loss: -27.0910 - acc: 0.01 - ETA: 23:55 - loss: -27.0945 - acc: 0.01 - ETA: 23:55 - loss: -27.1037 - acc: 0.01 - ETA: 23:54 - loss: -27.1165 - acc: 0.01 - ETA: 23:53 - loss: -27.1263 - acc: 0.01 - ETA: 23:52 - loss: -27.1173 -

...

55936/80190 [===================>..........] - ETA: 12:06 - loss: 43.7825 - acc: 0.549 - ETA: 12:05 - loss: 43.7822 - acc: 0.549 - ETA: 12:04 - loss: 43.7790 - acc: 0.549 - ETA: 12:03 - loss: 43.7793 - acc: 0.549 - ETA: 12:03 - loss: 43.7738 - acc: 0.550 - ETA: 12:02 - loss: 43.7729 - acc: 0.550 - ETA: 12:01 - loss: 43.7791 - acc: 0.549 - ETA: 12:00 - loss: 43.7833 - acc: 0.549 - ETA: 12:00 - loss: 43.7933 - acc: 0.549 - ETA: 11:59 - loss: 43.7979 - acc: 0.549 - ETA: 11:58 - loss: 43.7979 - acc: 0.549 - ETA: 11:57 - loss: 43.7979 - acc: 0.549 - ETA: 11:57 - loss: 43.8011 - acc: 0.549 - ETA: 11:56 - loss: 43.7992 - acc: 0.549 - ETA: 11:55 - loss: 43.8169 - acc: 0.549 - ETA: 11:54 - loss: 43.8166 - acc: 0.549 - ETA: 11:54 - loss: 43.8073 - acc: 0.549 - ETA: 11:53 - loss: 43.7993 - acc: 0.549 - ETA: 11:52 - loss: 43.8006 - acc: 0.549 - ETA: 11:51 - loss: 43.8006 - acc: 0.549 - ETA: 11:50 - loss: 43.8025 - acc: 0.549 - ETA: 11:50 - loss: 43.8028 - acc: 0.549 - ETA: 11:49 - loss: 43.7993 - 

62464/80190 [======================>.......] - ETA: 9:36 - loss: 43.9279 - acc: 0.54 - ETA: 9:35 - loss: 43.9258 - acc: 0.54 - ETA: 9:35 - loss: 43.9172 - acc: 0.54 - ETA: 9:34 - loss: 43.9128 - acc: 0.54 - ETA: 9:33 - loss: 43.9130 - acc: 0.54 - ETA: 9:32 - loss: 43.9110 - acc: 0.54 - ETA: 9:32 - loss: 43.9092 - acc: 0.54 - ETA: 9:31 - loss: 43.9040 - acc: 0.54 - ETA: 9:30 - loss: 43.9016 - acc: 0.54 - ETA: 9:29 - loss: 43.8981 - acc: 0.54 - ETA: 9:29 - loss: 43.8952 - acc: 0.54 - ETA: 9:28 - loss: 43.9012 - acc: 0.54 - ETA: 9:27 - loss: 43.9034 - acc: 0.54 - ETA: 9:26 - loss: 43.8963 - acc: 0.54 - ETA: 9:26 - loss: 43.9042 - acc: 0.54 - ETA: 9:25 - loss: 43.8990 - acc: 0.54 - ETA: 9:24 - loss: 43.9041 - acc: 0.54 - ETA: 9:23 - loss: 43.9009 - acc: 0.54 - ETA: 9:23 - loss: 43.8966 - acc: 0.54 - ETA: 9:22 - loss: 43.8963 - acc: 0.54 - ETA: 9:21 - loss: 43.9005 - acc: 0.54 - ETA: 9:20 - loss: 43.9019 - acc: 0.54 - ETA: 9:19 - loss: 43.9018 - acc: 0.54 - ETA: 9:19 - loss: 43.9029 - acc: 

68992/80190 [========================>.....] - ETA: 7:01 - loss: 43.9289 - acc: 0.54 - ETA: 7:00 - loss: 43.9260 - acc: 0.54 - ETA: 6:59 - loss: 43.9239 - acc: 0.54 - ETA: 6:59 - loss: 43.9176 - acc: 0.54 - ETA: 6:58 - loss: 43.9119 - acc: 0.54 - ETA: 6:57 - loss: 43.9070 - acc: 0.54 - ETA: 6:56 - loss: 43.9041 - acc: 0.54 - ETA: 6:55 - loss: 43.9059 - acc: 0.54 - ETA: 6:55 - loss: 43.9107 - acc: 0.54 - ETA: 6:54 - loss: 43.9099 - acc: 0.54 - ETA: 6:53 - loss: 43.9099 - acc: 0.54 - ETA: 6:52 - loss: 43.9106 - acc: 0.54 - ETA: 6:52 - loss: 43.9169 - acc: 0.54 - ETA: 6:51 - loss: 43.9166 - acc: 0.54 - ETA: 6:50 - loss: 43.9148 - acc: 0.54 - ETA: 6:49 - loss: 43.9083 - acc: 0.54 - ETA: 6:49 - loss: 43.9063 - acc: 0.54 - ETA: 6:48 - loss: 43.9052 - acc: 0.54 - ETA: 6:47 - loss: 43.9069 - acc: 0.54 - ETA: 6:46 - loss: 43.9128 - acc: 0.54 - ETA: 6:46 - loss: 43.9148 - acc: 0.54 - ETA: 6:45 - loss: 43.9214 - acc: 0.54 - ETA: 6:44 - loss: 43.9274 - acc: 0.54 - ETA: 6:43 - loss: 43.9141 - acc: 

75520/80190 [===========================>..] - ETA: 4:25 - loss: 43.8788 - acc: 0.54 - ETA: 4:25 - loss: 43.8848 - acc: 0.54 - ETA: 4:24 - loss: 43.8845 - acc: 0.54 - ETA: 4:23 - loss: 43.8885 - acc: 0.54 - ETA: 4:22 - loss: 43.8896 - acc: 0.54 - ETA: 4:22 - loss: 43.8854 - acc: 0.54 - ETA: 4:21 - loss: 43.8816 - acc: 0.54 - ETA: 4:20 - loss: 43.8790 - acc: 0.54 - ETA: 4:19 - loss: 43.8776 - acc: 0.54 - ETA: 4:19 - loss: 43.8739 - acc: 0.54 - ETA: 4:18 - loss: 43.8720 - acc: 0.54 - ETA: 4:17 - loss: 43.8724 - acc: 0.54 - ETA: 4:16 - loss: 43.8689 - acc: 0.54 - ETA: 4:15 - loss: 43.8682 - acc: 0.54 - ETA: 4:15 - loss: 43.8703 - acc: 0.54 - ETA: 4:14 - loss: 43.8753 - acc: 0.54 - ETA: 4:13 - loss: 43.8660 - acc: 0.54 - ETA: 4:12 - loss: 43.8644 - acc: 0.54 - ETA: 4:12 - loss: 43.8683 - acc: 0.54 - ETA: 4:11 - loss: 43.8725 - acc: 0.54 - ETA: 4:10 - loss: 43.8754 - acc: 0.54 - ETA: 4:09 - loss: 43.8782 - acc: 0.54 - ETA: 4:09 - loss: 43.8796 - acc: 0.54 - ETA: 4:08 - loss: 43.8793 - acc: 

80190/80190 [==============================] - ETA: 1:50 - loss: 43.9171 - acc: 0.54 - ETA: 1:49 - loss: 43.9213 - acc: 0.54 - ETA: 1:48 - loss: 43.9213 - acc: 0.54 - ETA: 1:48 - loss: 43.9236 - acc: 0.54 - ETA: 1:47 - loss: 43.9280 - acc: 0.54 - ETA: 1:46 - loss: 43.9277 - acc: 0.54 - ETA: 1:45 - loss: 43.9273 - acc: 0.54 - ETA: 1:45 - loss: 43.9319 - acc: 0.54 - ETA: 1:44 - loss: 43.9297 - acc: 0.54 - ETA: 1:43 - loss: 43.9318 - acc: 0.54 - ETA: 1:42 - loss: 43.9341 - acc: 0.54 - ETA: 1:42 - loss: 43.9272 - acc: 0.54 - ETA: 1:41 - loss: 43.9274 - acc: 0.54 - ETA: 1:40 - loss: 43.9236 - acc: 0.54 - ETA: 1:39 - loss: 43.9188 - acc: 0.54 - ETA: 1:39 - loss: 43.9171 - acc: 0.54 - ETA: 1:38 - loss: 43.9143 - acc: 0.54 - ETA: 1:37 - loss: 43.9149 - acc: 0.54 - ETA: 1:36 - loss: 43.9132 - acc: 0.54 - ETA: 1:35 - loss: 43.9129 - acc: 0.54 - ETA: 1:35 - loss: 43.9127 - acc: 0.54 - ETA: 1:34 - loss: 43.9145 - acc: 0.54 - ETA: 1:33 - loss: 43.9107 - acc: 0.54 - ETA: 1:32 - loss: 43.9075 - acc: 

In [12]:
# evaluate model on test set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 44.70917298773278
Test accuracy: 0.539356528247647
